In [151]:
# Install libraries on first run
#! pip install -q ipynb fastai pathlib import_ipynb

In [152]:
from fastai.tabular.all import *
from pathlib import Path
import import_ipynb
import ipynb
import stockFetcher

In [153]:
testFolder = Path.cwd().parent / 'TestData'
modelFolder = Path.cwd().parent.parent / 'TrainedModels'

In [154]:
print('Model files in modelFolder:')
for file in modelFolder.glob('*.pkl'):
    print(file.name)

Model files in modelFolder:
countryGuesser.pkl
stockScreenerV2.1.pkl
stockScreenerV2.0.pkl
stockScreenerV1.5.pkl
animalClassifier.pkl
stockScreenerV1.4.pkl
stockScreenerV1.6.pkl
leagueChampionClassifier.pkl
stockScreenerV1.7.pkl
politicianClassifier.pkl
stockScreenerV1.3.pkl
stockScreenerV3.0.pkl
stockScreenerV1.2.pkl
stockScreenerV1.0.pkl
stockScreenerV1.1.pkl
test.pkl
stockScreenerV1.9.pkl
stockScreenerV1.8.pkl
stockScreenerV1.10.pkl


In [155]:
evaluations = pd.read_csv(modelFolder / 'modelEvaluations.csv')
bestModel = evaluations.sort_values('MAE', ascending=True).iloc[0]
bestModel.head()

Model Name    stockScreenerV2.1
Timestamp      2025-01-19 14:33
MAE                       0.205
RMSE                      0.284
R2                       -0.331
Name: 12, dtype: object

## Load model

In [156]:
modelName = f"{bestModel['Model Name']}.pkl" # Change this if you want to try other models
learn = load_learner(modelFolder / modelName)

## Predictions

In [157]:
predictionTarget = 'NVO' # 'Best' for the best performing tickers 

if predictionTarget == 'Best':
    adr_df = pd.read_csv(testFolder / 'tickers.csv')
    symbols = adr_df['Ticker'].tolist()
    test_df = [stockFetcher.getTickerData(symbol) for symbol in symbols]
    test_df = pd.concat(test_df, ignore_index=True)  # Concatenate list of DataFrames into a single DataFrame

else:
    test_df = stockFetcher.getTickerData(predictionTarget)

# Ensure test_df is a DataFrame
if isinstance(test_df, dict):
    test_df = pd.DataFrame([test_df])

dl = learn.dls.test_dl(test_df)
test_df.head()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  to[n].fillna(self.na_dict[n], inplace=True)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/fastai/tabular/core.py:314: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermed

,Open,High,Low,Close,Volume,Dividends,Stock Splits,EV/EBIT,Market Cap,ROIC,Industry
0,79.940002,80.879997,78.169998,78.690002,21243900,0.0,0.0,6.08427,2.648540e+11,0.129843,Drug Manufacturers - General


In [159]:
if predictionTarget == 'Best':
    prediction = learn.get_preds(dl=dl)
    sorted_predictions = sorted(zip(symbols, prediction[0]), key=lambda x: x[1], reverse=True)
    print(f"Prediction for 1 year change:")
    for symbol, pred in sorted_predictions:
        company_name = adr_df[adr_df['Ticker'] == symbol]['Company'].values[0]
        print(f"{symbol} ({company_name}): {pred[0].item() * 100:.2f}%")
else:
    prediction = learn.get_preds(dl=dl)
    company_name = adr_df[adr_df['Ticker'] == predictionTarget]['Company'].values[0]
    print(f"{predictionTarget} ({company_name}):")
    print(f"{prediction[0][0][0].item() * 100:.2f}%")

print("Free money?!")

IndexError: index 0 is out of bounds for axis 0 with size 0